## Globale Temperaturänderungen und Gefahren

### Teil 4
- Kumulative Aggregation der Temperaturdaten

---

In [33]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import YearEnd
from pandas.api.types import CategoricalDtype

import dash
from dash import Dash, dcc, html, Input, Output, callback, callback_context

import plotly.figure_factory as ff


## 1. CSV einlesen
Zuvor gespeicherte CSV wird ins DataFrame geladen:
- clim_change_celsius_df.csv `celsius_df`

In [34]:
""" Dauer etwa 18 Sekunden """

# Einlesen des Temperatur-DataFrames --> ca. 17 Sekunden
celsius_df = pd.read_csv('clim_change_celsius_df.csv', parse_dates=['dt'],
                 usecols=['dt', 'AverageTemperature', 'City', 'plus_code'],
                 dtype={'AverageTemperature': 'float32',
                        'City': 'string',
                        'plus_code': 'string'
                       }
                )
celsius_df.info(show_counts=True, memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8235082 entries, 0 to 8235081
Data columns (total 4 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   dt                  8235082 non-null  datetime64[ns]
 1   AverageTemperature  8235082 non-null  float32       
 2   City                8235082 non-null  string        
 3   plus_code           8235082 non-null  string        
dtypes: datetime64[ns](1), float32(1), string(2)
memory usage: 1004.6 MB


## 2. Kumulierte Jahresaggregation
- Temperaturen werden auf Jahre aggregiert (min/max/avg)
  - `c_avg` Jahresdurschnitt
- Aggregationen mit Kumulation aller vorangegangenen Jahre
  - `c_cum_avg` Kumulativer Durschnitt
  - `c_cum_min` Kumulatives Minimum
  - `c_cum_max` Kumulatives Maximum
  - `c_cum_diff` Delta aus Kum. Max. & Kum. Min.

---
### 2.1. GroupBy-Objekt
Ein GroupBy-Objekt ist eine Zwischenstruktur in Pandas, die entsteht\
wenn ein DataFrame mit `groupby()` nach bestimmten Schlüsseln gruppiert wird.
- Split: Daten werden angegebenen Kriterien in Gruppen aufgeteilt
- Apply: Auf jede Gruppe kann eine Funktion angewendet werden (z.B. Aggregation)
- Combine: Ergebnisse werden wieder zu einem neuen Objekt zusammengeführt
- Man kann über die Gruppen iterieren und erhält jeweils den Gruppenschlüssel und die zugehörigen Daten
````python
grouped_year = celsius_df.groupby(['City', 'plus_code'])
````

### 2.2. Transform-Methode
Mit `transform()` wird eine Funktion auf jede Gruppe angewendet und ein Ergebnis mit Länge der Originalgruppe zurückgegeben.\
Im Gegensatz zu agg() behält es die ursprüngliche DataFrame-Struktur bei.

### 2.3. expanding()-Funktion
`x.expanding()` erzeugt ein sich erweiterndes Fenster:
- Startet mit der ersten Beobachtung
- Vergrößert sich schrittweise um je eine Beobachtung
- Beinhaltet immer alle vorherigen Werte + aktuellen Wert
- Entspricht SQL Fensterfunkion: OVER (PARTITION BY ...)

````python
grouped_year['c_avg'].transform(lambda x: x.expanding().mean())
````

---
### 2.4. gruppieren & aggregieren

In [35]:
celsius_year_df = (
    celsius_df
    .assign(year=lambda x: x['dt'].dt.year)
    .sort_values(['City', 'plus_code', 'dt'])
    .groupby(['City', 'plus_code', 'year'], as_index=False, observed=True)
    .agg(
        c_avg=('AverageTemperature', 'mean'),
        c_min=('AverageTemperature', 'min'),
        c_max=('AverageTemperature', 'max')
    )
    .sort_values(['City', 'year'])
)


In [36]:
# Kumulierte Aggregation für Jahre
grouped_year = celsius_year_df.groupby(['City', 'plus_code'])
celsius_year_df['c_cum_avg'] = grouped_year['c_avg'].transform(lambda x: x.expanding().mean()).astype('float32')
celsius_year_df['c_cum_min'] = grouped_year['c_min'].transform(lambda x: x.expanding().min()).astype('float32')
celsius_year_df['c_cum_max'] = grouped_year['c_max'].transform(lambda x: x.expanding().max()).astype('float32')
del grouped_year

# Temperaturdifferenz (c_cum_max - c_cum_min)
celsius_year_df['c_cum_diff'] = celsius_year_df['c_cum_max'] - celsius_year_df['c_cum_min']

# Ausgabe
celsius_year_df.info()
print('')
print(celsius_year_df[celsius_year_df.City.isin(['Ternate'])].head(10).to_string(index=False))

<class 'pandas.core.frame.DataFrame'>
Index: 693249 entries, 0 to 693248
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   City        693249 non-null  string 
 1   plus_code   693249 non-null  string 
 2   year        693249 non-null  int32  
 3   c_avg       693249 non-null  float32
 4   c_min       693249 non-null  float32
 5   c_max       693249 non-null  float32
 6   c_cum_avg   693249 non-null  float32
 7   c_cum_min   693249 non-null  float32
 8   c_cum_max   693249 non-null  float32
 9   c_cum_diff  693249 non-null  float32
dtypes: float32(7), int32(1), string(2)
memory usage: 37.0 MB

   City plus_code  year     c_avg     c_min     c_max  c_cum_avg  c_cum_min  c_cum_max  c_cum_diff
Ternate 6QG9RQ29+  1880 25.926332 25.201000 26.612000  25.926332     25.201     26.612    1.410999
Ternate 6QG9RQ29+  1881 25.827375 25.295000 26.173000  25.876854     25.201     26.612    1.410999
Ternate 6QG9RQ29+  1882 26.159

### 2.5. CSV Export
DataFrame celsius_year_df wird als CSV gespeichert.
- clim_change_celsius_year_df.csv

In [37]:
# Dataframe als CSV speichern
celsius_year_df.to_csv('clim_change_celsius_year_df.csv', index=False)
del celsius_year_df

## 3. Nach Jahreszeiten kumulieren
- Temperaturen werden auf Jahreszeiten aggregiert (min/max/avg)
  - `c_avg` Jahreszeitendurschnitt
- Aggregationen mit Kumulation aller vorangegangenen Jahre
  - `c_cum_avg` Kumulativer Durschnitt
  - `c_cum_min` Kumulatives Minimum
  - `c_cum_max` Kumulatives Maximum
  - `c_cum_diff` Delta aus Kum.-Max. & Kum.-Min.

---
Meteorologisch werden Jahreszeiten aufgeteilt nach:
- Frühling (März, April, Mai)
- Sommer (Juni, Juli, August)
- Herbst (September, Oktober, November)
- Winter (Dezember, Januar, Februar)

Diese Einteilung basiert auf einem internationalen Abkommen und dient der Vereinfachung der Erfassung und Verarbeitung von Wetterdaten.

---

### 3.1. np.where()

In [38]:
# # ca. 71 Sekunden
# def assign_season(row):
#     if 3 <= row.month <= 5:
#         return 'spring'
#     elif 6 <= row.month <= 8:
#         return 'summer'
#     elif 9 <= row.month <= 11:
#         return 'autumn'
#     else:
#         return 'winter'

# celsius_df = (
#     celsius_df
#     .assign(
#         month=lambda x: x['dt'].dt.month,
#         season_year=lambda x: np.where(
#             x['month'] == 12,
#             x['dt'].dt.year + 1,
#             x['dt'].dt.year
#         ),
#         season=lambda x: x.apply(assign_season, axis=1)
#     )
# )

# import time
# start = time.perf_counter()
# end = time.perf_counter()
# print(f"Ausführungszeit: {end - start:.3f} Sekunden")

### 3.2. pd.offsets.YearEnd
Verwendung von `pd.offsets.YearEnd` für Winter-Jahreszuordnung
````python
from pandas.tseries.offsets import YearEnd
````

Vorteile der YearEnd()-Methode gegenüber np.where()
- **6x schneller als np.where() bei den hier vorhandenen 8.2M Zeilen**
- Robust gegenüber Zeitzonen und ungewöhnlichen Datumsformaten
- Nutzt interne C-optimierte Pandas-Routinen
- Vermeidet Konvertierung zwischen Pandas/NumPy-Datentypen
- **~15% geringere Speicherverwendung bei großen Datenmengen**
- Arbeitet direkt auf Pandas-DateTime-Objekten
- Keine temporären Arrays wie bei np.where()

---
### 3.3. pd.Categorical
`pd.Categorical` ist ein Pandas Datentyp für Spalten mit einer festen Anzahl möglicher Werte (z.B. Jahreszeiten).
Statt jeden Wert als String zu speichern, werden die Werte intern als Integer-Codes auf eine Liste der Kategorien gemappt.
````python
from pandas.api.types import CategoricalDtype
````

**Weniger Speicherverbrauch** bei großen Datenmengen mit sich wiederholenden Werten (z.B. Millionen Zeilen, aber nur 4 Jahreszeiten).\
**Schnellere Berechnungen** von Gruppierungen, Sortierungen und Aggregationen sind mit kategorischen Daten deutlich schneller.\
Es können nur Werte verwendet werden, die in den Kategorien definiert sind. Falsche Werte führen zu Fehlern.\
Mit ordered=True kann eine logische Reihenfolge der Kategorien festgelegt werden (z.B. für Zeitreihenvergleiche).
- Das Argument `categories` legt explizit fest, welche Werte als gültige Kategorien in welcher Reihenfolge zulässig sind
  - Andere Werte werden als NaN behandelt
  - `ordered=True` bestimmt die Reihenfolge in categories

---

In [39]:
""" Dauer etwa 19 Sekunden """

# Jahreszeiten-Bereiche definieren
conditions = [
    celsius_df['dt'].dt.month.between(3, 5),
    celsius_df['dt'].dt.month.between(6, 8),
    celsius_df['dt'].dt.month.between(9, 11)
]

# Jahreszeiten zuordnen mit pd.Categorical und expliziter Reihenfolge
celsius_df['season'] = pd.Categorical(
    values=np.select(conditions, ['spring', 'summer', 'autumn'], default='winter'),
    categories=['winter', 'spring', 'summer', 'autumn'],
    ordered=True
)

# celsius_df gruppieren & aggregieren nach celsius_season_df --> ca. 7 Sekunden
celsius_season_df = (
    celsius_df
    .sort_values(['City', 'plus_code', 'dt'])
    .groupby(['City', 'plus_code', 'dt', 'season'], as_index=False, observed=True)
    .agg(c_avg=('AverageTemperature', 'mean'))
    .sort_values(['City', 'season'])
)
del celsius_df

# Winter-Jahreszuordnung --> ca. 10 Sekunden
celsius_season_df['season_year'] = (
    celsius_season_df['dt']
    .apply(lambda d: (d + YearEnd(n=1)).year if d.month == 12 else d.year)
)

# Ausgabe
celsius_season_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8235082 entries, 8 to 8235072
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   City         string        
 1   plus_code    string        
 2   dt           datetime64[ns]
 3   season       category      
 4   c_avg        float32       
 5   season_year  int64         
dtypes: category(1), datetime64[ns](1), float32(1), int64(1), string(2)
memory usage: 353.4 MB


### 3.4. gruppieren & aggregieren

In [40]:
""" Dauer etwa 10 Sekunden """

# celsius_season_df gruppieren & aggregieren --> ca. 5 Sekunden
celsius_season_df = (
    celsius_season_df
    .sort_values(['City', 'plus_code', 'dt'])
    .groupby(['City', 'plus_code', 'season_year', 'season'], as_index=False, observed=True)
    .agg(c_avg=('c_avg', 'mean'))
    .sort_values(['City', 'season'])
)

# Kumulierte Aggregetion für Jahreszeiten
grouped_season = celsius_season_df.groupby(['City', 'plus_code', 'season'], observed=True)
celsius_season_df['c_cum_avg'] = grouped_season['c_avg'].transform(lambda x: x.expanding().mean()).astype('float32')
celsius_season_df['c_cum_min'] = grouped_season['c_avg'].transform(lambda x: x.expanding().min()).astype('float32')
celsius_season_df['c_cum_max'] = grouped_season['c_avg'].transform(lambda x: x.expanding().max()).astype('float32')
del grouped_season

# Temperaturdifferenz (c_cum_max - c_cum_min)
celsius_season_df['c_cum_diff'] = celsius_season_df['c_cum_max'] - celsius_season_df['c_cum_min']

# Ausgabe
celsius_season_df.info()
print('')
print(celsius_season_df[celsius_season_df.City.isin(['Ternate'])].head(12).to_string(index=False))

end = time.perf_counter()
print(f'Ausführungszeit: {end - start:.3f} Sekunden')

<class 'pandas.core.frame.DataFrame'>
Index: 2759692 entries, 1 to 2759688
Data columns (total 9 columns):
 #   Column       Dtype   
---  ------       -----   
 0   City         string  
 1   plus_code    string  
 2   season_year  int64   
 3   season       category
 4   c_avg        float32 
 5   c_cum_avg    float32 
 6   c_cum_min    float32 
 7   c_cum_max    float32 
 8   c_cum_diff   float32 
dtypes: category(1), float32(5), int64(1), string(2)
memory usage: 139.5 MB

   City plus_code  season_year season     c_avg  c_cum_avg  c_cum_min  c_cum_max  c_cum_diff
Ternate 6QG9RQ29+         1880 winter 25.415001  25.415001  25.415001  25.415001    0.000000
Ternate 6QG9RQ29+         1881 winter 25.535999  25.475500  25.415001  25.535999    0.120998
Ternate 6QG9RQ29+         1882 winter 26.384333  25.778444  25.415001  26.384333    0.969332
Ternate 6QG9RQ29+         1883 winter 25.610666  25.736500  25.415001  26.384333    0.969332
Ternate 6QG9RQ29+         1884 winter 25.418665  25.67

### 3.5. CSV Export
DataFrame celsius_season_df wird als CSV gespeichert.
- clim_changecelsius_season_df.csv

In [41]:
""" Dauer etwa 14 Sekunden """

# Dataframe als CSV speichern
celsius_season_df.to_csv('clim_change_celsius_season_df.csv', index=False)
del celsius_season_df